In [1]:
from transformers import pipeline
import accelerate
import time

In [ ]:
from 

In [2]:

#hf_model = pipeline(
#    "text-generation", model="cerebras/Cerebras-GPT-590M", max_new_tokens=200
#)

In [3]:
from jsonformer import Jsonformer
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
#from langchain.llms import HuggingFacePipeline
#import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

model_id = 'google/flan-t5-large'# go for a smaller model if you dont have the VRAM
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=False)


In [24]:
if True:
    model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v2-7b")
    tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-7b")
elif False:
    model_name = 'tiiuae/falcon-7b'
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [66]:
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 4096)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)

In [40]:
json_schema = {
    "type": "object",
    "properties": {
        "origin": {
            "type": "string",
            "description": "Pickup City,State of load"
        },
        "destination": {
            "type": "string",
            "description": "Dropoff City,State of load"
        },
        "equipment": {
            "type": "string",
            "enum": ['Van', 'Reefer', 'Flatbed'],
            "description": "The type of truckload often Van Reefer or Flatbed"
        },
        "weight": {
            "type": "number",
            "description": "Total weight of the load"
        },
        "weight_unit": {
            "type": "string",
            "description": "measurement unit of weight"},
        "hot": {
            "type": "boolean",
            "default": False,
            "description": "if the messages has HOT in it",
        },
        "team": {
            "type": "boolean",
            "default": False,
            "description": "If message has word TEAM in it"
        },
        "latefee": {
            "type": "number",
            "required": False,
            "description": "dollar amount fee if a load is late",
        },
        "dropoff time": {
            "type": "string",
            "required": False,
            "description": "Time load picksup HH:MM"
        }
    }
}

In [33]:
prompt_1 = "Van load going from Chicago,IL To Dallas,TX 43000 lbs"
prompt_2 = "Reefer load from Los Angeles,CA to Las Vegas,NV 30000 lbs"
prompt_3 = "Flatbed load from Los Angeles,CA to Las Vegas,NV 22200 lbs"

In [34]:
def process_raw_text(model, tokenizer, json_schema, raw_text):
    prompt_template = "Your job is to extract information from the given message and populate a json sctructure using keywords from the message. The message: {}"

    jsonformer = Jsonformer(model, tokenizer, json_schema, prompt_template.format(raw_text))
    generated_data = jsonformer()
    
    return generated_data

In [35]:
#prompt_template = "Your job is to extract information from the given message and populate a json sctructure using keywords from the message. The message: {}"

In [36]:
process_raw_text(model, tokenizer, json_schema, prompt_2)

{'origin': 'Los Angeles,CA',
 'destination': 'Las Vegas,NV',
 'equipment': 'Van',
 'weight': 30000.0,
 'weight_unit': 'ton',
 'hot': True,
 'team': True,
 'latefee': 500.0,
 'dropoff time': '10:00 AM'}

In [13]:
#prompt = "Bob is a 19 year old student taking CS101 and Advanced Algorithms. Generate the following schema:"
jsonformer = Jsonformer(model, tokenizer, json_schema, prompt_template.format(prompt_1))
generated_data = jsonformer()

print(generated_data)

NameError: name 'prompt_template' is not defined

## Make a test data set - Logistics data
- Equipment: Van/Reefer/Flatbed
- Origin/Destination: random selection from list of cities
- Weight: random(0,45000)
- other attributes:
  - HOT
  - $X Late fee
  - Team load
  - Dropoff time HH:00


In [67]:
import pandas as pd
from src.utils import make_random_loads

In [15]:
df_cities = pd.read_csv("static/uscities.csv")

In [16]:
#df_cities.iloc[random.randint(0,df_cities.shape[0])]

In [56]:
import random

def make_random_load(p_hot=1.0, p_team=1.0, p_dropoff=1.0, p_latefee=0.25):
    out = {}

    out['Equipment'] = random.choice(['Van', 'Reefer', 'Flatbed'])
    out['Weight'] = random.randint(0,45000)

    # Origin / Destination
    origin = df_cities.iloc[random.randint(0,df_cities.shape[0])]
    dest = df_cities.iloc[random.randint(0,df_cities.shape[0])]

    out['Origin'] = f"{origin['city']}, {origin['state_id']}"
    out['Destination'] = f"{dest['city']}, {dest['state_id']}"

    
    # Optional fields
    if random.random() < p_hot:
        out['Hot'] = True #random.choice(['HOT', 'HOT LOAD'])
        
    if random.random() < p_team:
        out['Team'] = True #random.choice(['TEAM', 'TEAM LOAD'])
        
    if random.random() < p_dropoff:
        out['Dropoff'] = f"{random.randint(6,23):02d}:00 Dropoff" 
        
    if random.random() < p_latefee:
        out['Latefee'] = f"${random.randint(50,250+1)} late fee"

    return out

# Make load into a string
load = make_random_load()

load

{'Equipment': 'Flatbed',
 'Weight': 39086,
 'Origin': 'Thayer, NE',
 'Destination': 'Holley, NY',
 'Hot': True,
 'Team': True,
 'Dropoff': '23:00 Dropoff'}

In [57]:
#process_raw_text(model, tokenizer, json_schema, load)

In [58]:
loads = [make_random_load() for _ in range(10)]

In [64]:
extra_cols = ['Team', 'Hot', 'Dropoff', 'Latefee']


load_strs = []
loads = [make_random_load() for _ in range(20)]

for load in loads:
    details = []
    
    for ec in extra_cols:
        if ec in load:
            if ec == 'Hot':
                s = random.choice(['HOT', 'HOT LOAD'])
            elif ec == 'Team':
                s = random.choice(['TEAM', 'TEAM LOAD'])
            else:
                s = load[ec]
            details.append(s)
    
    load_str = f"{load['Equipment']} load from {load['Origin']} to {load['Destination']} total weight {load['Weight']} lbs {' '.join(details)}"

    load_strs.append(load_str)

load

{'Equipment': 'Reefer',
 'Weight': 8048,
 'Origin': 'Middletown, OH',
 'Destination': 'Mariposa, CA',
 'Hot': True,
 'Team': True,
 'Dropoff': '19:00 Dropoff',
 'Latefee': '$131 late fee'}

In [68]:
loads, load_strs = make_random_loads(3)

In [70]:
llm_out = []

for load_real, load_str in zip(loads, load_strs):
    _start_time = time.time()

    print(f"Input: {load_str}")
    
    print("LLM:")
    
    pred = process_raw_text(model, tokenizer, json_schema, load_str)
    print(pred)

    llm_out.append(pred)

    print(f"elapsed: {time.time() - _start_time}s")
    
    print("-"*40)
    print()

Input: Flatbed load from Tillatoba, MS to Matlock, IA total weight 35078 lbs TEAM LOAD HOT 16:00 Dropoff
LLM:
{'origin': 'Tillatoba, MS', 'destination': 'Matlock, IA', 'equipment': 'Flatbed', 'weight': 35078.0, 'weight_unit': 'lbs', 'hot': True, 'team': True, 'latefee': 1000.0, 'dropoff time': '16:00'}
elapsed: 89.8296251296997s
----------------------------------------

Input: Van load from Ulen, MN to Ernstville, MD total weight 33089 lbs TEAM LOAD HOT LOAD 17:00 Dropoff
LLM:
{'origin': 'Ulen,MN', 'destination': 'Ernstville,MD', 'equipment': 'Van', 'weight': 33089.0, 'weight_unit': 'lbs', 'hot': True, 'team': True, 'latefee': 1000.0, 'dropoff time': '17:00'}
elapsed: 96.23345589637756s
----------------------------------------

Input: Reefer load from Bridgman, MI to Syracuse, KS total weight 10605 lbs TEAM LOAD HOT LOAD 12:00 Dropoff $58 late fee
LLM:
{'origin': 'Bridgman, MI', 'destination': 'Syracuse, KS', 'equipment': 'Van', 'weight': 10605.0, 'weight_unit': 'lbs', 'hot': True, 'te

In [76]:
df_pred = pd.DataFrame(llm_out)

In [75]:
df_real = pd.DataFrame(loads)

In [83]:
df_real.columns.str.lower()

Index(['equipment', 'weight', 'origin', 'destination', 'hot', 'team',
       'dropoff', 'latefee'],
      dtype='object')

In [81]:
df_pred.columns

Index(['origin', 'destination', 'equipment', 'weight', 'weight_unit', 'hot',
       'team', 'latefee', 'dropoff time'],
      dtype='object')

In [84]:
df_pred[df_real.columns.str.lower()]

KeyError: "['dropoff'] not in index"

In [58]:
#random.random() < p_hot:

SyntaxError: invalid syntax (2915883945.py, line 1)

## Marketing stuff

### Input

In [191]:

print("\n".join(load_strs[:3] + ['...'] + load_strs[-1:]))

Van load from Banks, ID to Danville, KY total weight 14153 lbs $50 late fee
Flatbed load from North Puyallup, WA to Bay Hill, FL total weight 5226 lbs TEAM
Van load from Webb City, MO to Roodhouse, IL total weight 17854 lbs TEAM LOAD
...
Reefer load from Mio, MI to Sarahsville, OH total weight 23763 lbs 06:00 Dropoff


### Schema

In [192]:
json_schema

{'type': 'object',
 'properties': {'origin': {'type': 'string',
   'description': 'Pickup City,State of load'},
  'destination': {'type': 'string',
   'description': 'Dropoff City,State of load'},
  'equipment': {'type': 'string',
   'enum': ['Van', 'Reefer', 'Flatbed'],
   'description': 'The type of truckload, often Van, Reefer, or Flatbed'},
  'weight': {'type': 'number', 'description': 'Total weight of the load'},
  'weight_unit': {'type': 'string',
   'description': 'measurement unit of weight'},
  'hot': {'type': 'boolean', 'description': 'HOT load, hot'},
  'team': {'type': 'boolean',
   'description': 'If a load requires two drivers. Ex: TEAM, TEAM load'},
  'latefee': {'type': 'number',
   'required': False,
   'description': 'dollar amount fee if a load is late'},
  'dropoff time': {'type': 'string',
   'required': False,
   'description': 'Time load picksup HH:MM'}}}

## Schema - Human readable

In [193]:
for key,val in json_schema.get('properties').items():
    optional = True if (val.get('default') is not None or val.get('required') == False) else False
    
    term_base = f"{key} - {val.get('type')}{' (Optional)' if optional else ''}: "


    desc = val.get('description') or ""
    if val.get('enum'):
        desc = desc + '\n    '+ f'Must be one of: {val.get("enum")}'
    
    print(f"{term_base} {desc}")

origin - string:  Pickup City,State of load
destination - string:  Dropoff City,State of load
equipment - string:  The type of truckload, often Van, Reefer, or Flatbed
    Must be one of: ['Van', 'Reefer', 'Flatbed']
weight - number:  Total weight of the load
weight_unit - string:  measurement unit of weight
hot - boolean:  HOT load, hot
team - boolean:  If a load requires two drivers. Ex: TEAM, TEAM load
latefee - number (Optional):  dollar amount fee if a load is late
dropoff time - string (Optional):  Time load picksup HH:MM


## Output

In [194]:
test_input = list(zip(loads, load_strs))

In [195]:
llm_out = []

for load_real, load_str in (test_input[:3] + test_input[-1:]):
    _start_time = time.time()

    print(f"Input: {load_str}")
    
    print("LLM:")
    
    pred = process_raw_text(model, tokenizer, json_schema, load_str)
    print(pred)

    llm_out.append(pred)

    print(f"elapsed: {time.time() - _start_time}s")
    
    print("-"*40)
    print()

Input: Van load from Banks, ID to Danville, KY total weight 14153 lbs $50 late fee
LLM:
{'origin': 'Banks,ID', 'destination': 'Danville,KY', 'equipment': 'Van', 'weight': 14153.0, 'weight_unit': 'lbs', 'hot': False, 'team': False, 'latefee': 50.0, 'dropoff time': 'HH:MM'}
elapsed: 104.97942781448364s
----------------------------------------

Input: Flatbed load from North Puyallup, WA to Bay Hill, FL total weight 5226 lbs TEAM
LLM:
{'origin': 'Puyallup, WA', 'destination': 'Bay Hill, FL', 'equipment': 'Van', 'weight': 5226.0, 'weight_unit': 'lbs', 'hot': False, 'team': False, 'latefee': 0.0, 'dropoff time': '11:00 AM'}
elapsed: 106.35575699806213s
----------------------------------------

Input: Van load from Webb City, MO to Roodhouse, IL total weight 17854 lbs TEAM LOAD
LLM:
{'origin': 'Webb City, MO', 'destination': 'Roodhouse, IL', 'equipment': 'Van', 'weight': 17854.0, 'weight_unit': 'lbs', 'hot': False, 'team': False, 'latefee': 0.0, 'dropoff time': '11:00 AM'}
elapsed: 92.693721

In [196]:
pd.DataFrame(llm_out)

,origin,destination,equipment,weight,weight_unit,hot,team,latefee,dropoff time
0,"Banks,ID","Danville,KY",Van,14153.0,lbs,False,False,50.0,HH:MM
1,"Puyallup, WA","Bay Hill, FL",Van,5226.0,lbs,False,False,0.0,11:00 AM
2,"Webb City, MO","Roodhouse, IL",Van,17854.0,lbs,False,False,0.0,11:00 AM
3,"Mio, MI","Sarahsville, OH",Van,23763.0,lbs,False,False,0.0,06:00


In [198]:
llm_out

[{'origin': 'Banks,ID',
  'destination': 'Danville,KY',
  'equipment': 'Van',
  'weight': 14153.0,
  'weight_unit': 'lbs',
  'hot': False,
  'team': False,
  'latefee': 50.0,
  'dropoff time': 'HH:MM'},
 {'origin': 'Puyallup, WA',
  'destination': 'Bay Hill, FL',
  'equipment': 'Van',
  'weight': 5226.0,
  'weight_unit': 'lbs',
  'hot': False,
  'team': False,
  'latefee': 0.0,
  'dropoff time': '11:00 AM'},
 {'origin': 'Webb City, MO',
  'destination': 'Roodhouse, IL',
  'equipment': 'Van',
  'weight': 17854.0,
  'weight_unit': 'lbs',
  'hot': False,
  'team': False,
  'latefee': 0.0,
  'dropoff time': '11:00 AM'},
 {'origin': 'Mio, MI',
  'destination': 'Sarahsville, OH',
  'equipment': 'Van',
  'weight': 23763.0,
  'weight_unit': 'lbs',
  'hot': False,
  'team': False,
  'latefee': 0.0,
  'dropoff time': '06:00'}]

In [197]:
pd.DataFrame(loads[:3] + loads[-1:])

,Equipment,Weight,Origin,Destination,Latefee,Team,Dropoff
0,Van,14153,"Banks, ID","Danville, KY",$50 late fee,NaN,NaN
1,Flatbed,5226,"North Puyallup, WA","Bay Hill, FL",NaN,TEAM,NaN
2,Van,17854,"Webb City, MO","Roodhouse, IL",NaN,TEAM LOAD,NaN
3,Reefer,23763,"Mio, MI","Sarahsville, OH",NaN,NaN,06:00 Dropoff


In [52]:
tokenizer

GPTNeoXTokenizerFast(name_or_path='databricks/dolly-v2-7b', vocab_size=50254, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['### End', '### Instruction:', '### Response:']}, clean_up_tokenization_spaces=True)

torch.Size([50280])